In [1]:
using CSV, DataFrames, Statistics, StatsBase, Distributions, JuMP, Gurobi, LinearAlgebra


#include("eval2.jl")
const GRB_ENV = Gurobi.Env()

Academic license - for non-commercial use only - expires 2022-09-11


Gurobi.Env(Ptr{Nothing} @0x00007fedf50cca00, false, 0)

In [13]:
import Pkg

include("src/algos/multi_armed_bandits.jl")
include("src/algos/passive_agressive.jl")
include("src/algos")
include("src/algos/OLS.jl")
include("src/algos/adaptive_linear_decision_rule.jl")
include("src/metrics.jl")
include("src/utils.jl")

prepare_data_from_X (generic function with 1 method)

In [295]:
id = "1494"
X_test_adaptive = CSV.read("data/data_M3F/data/M3F_data_"*id*".csv", DataFrame)
y_test = CSV.read("data/data_M3F/M3F_targets/M3F_target_"*id*".csv", DataFrame)
y_test = y_test[!, "target"]
X = Matrix(X_test_adaptive)

18×23 Matrix{Float64}:
  0.0  4230.0  4402.82  4627.23  …  4271.47  4266.0   4191.13  4430.88
  1.0  4230.0  4402.82  4633.89     4244.55  4187.79  4182.57  4430.88
  2.0  4230.0  4402.82  4640.55     4738.77  4798.35  4659.37  4430.88
  3.0  4230.0  4402.82  4647.21     4863.17  4837.15  4685.62  4430.88
  4.0  4230.0  4402.82  4653.87     4373.34  4235.88  4149.13  4430.88
  5.0  4230.0  4402.82  4660.53  …  4359.48  4551.98  4408.62  4430.88
  6.0  4230.0  4402.82  4667.19     4381.41  4390.0   4416.69  4430.88
  7.0  4230.0  4402.82  4673.85     4527.84  4533.21  4427.88  4430.88
  8.0  4230.0  4402.82  4680.51     5066.01  4811.54  4952.58  4430.88
  9.0  4230.0  4402.82  4687.17     5097.58  5297.14  5001.3   4430.88
 10.0  4230.0  4402.82  4693.83  …  4219.33  4112.09  4172.12  4430.88
 11.0  4230.0  4402.82  4700.49     4469.4   4539.43  4424.49  4430.88
 12.0  4230.0  4402.82  4707.15     4271.41  4299.88  4235.28  4430.88
 13.0  4230.0  4402.82  4713.81     4244.54  4220.88  

In [296]:
n, p = size(X)

#y_safi = (y .- mean(y[1:floor(Int, n/2),:], dims =1))./ std(y[1:floor(Int, n/2),:], dims=1);
split_index = floor(Int,n*0.5)

#X = (X .- mean(X[1:floor(Int, split_index),:], dims =1))./ std(X[1:floor(Int, split_index),:], dims=1)#[!,1]
y = y_test[:, 1];
mean_y = mean(y[1:floor(Int, split_index),:])
std_y = std(y[1:floor(Int, split_index),:])

X = (X .- mean_y)./std_y;
y = (y .- mean_y)./std_y;
println("Mean target: ", mean_y, " Std target: ", std_y)

Mean target: 6402.222222222223 Std target: 323.9512994949155


## Testing Linear Adaptive Rule

In [297]:
X_test_adaptive

,Column1,NAIVE2,SINGLE,HOLT,DAMPEN,WINTER,COMB S-H-D,B-J auto,AutoBox1
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0,4230.0,4402.82,4627.23,4517.2,4627.23,4515.75,4245.5,4317.84
2,1,4230.0,4402.82,4633.89,4516.58,4633.89,4517.76,4193.19,4317.84
3,2,4230.0,4402.82,4640.55,4516.48,4640.55,4519.95,4623.33,4860.0
4,3,4230.0,4402.82,4647.21,4516.46,4647.21,4522.16,4402.45,4317.84
5,4,4230.0,4402.82,4653.87,4516.46,4653.87,4524.38,4146.69,4317.84
6,5,4230.0,4402.82,4660.53,4516.46,4660.53,4526.6,4547.77,4317.84
7,6,4230.0,4402.82,4667.19,4516.46,4667.19,4528.82,4536.14,4317.84
8,7,4230.0,4402.82,4673.85,4516.46,4673.85,4531.04,4274.57,4317.84
9,8,4230.0,4402.82,4680.51,4516.46,4680.51,4533.26,4797.71,5330.0


In [298]:
y_test

18-element Vector{Float64}:
 6400.0
 6580.0
 6120.0
 6580.0
 6040.0
 6480.0
 5880.0
 6860.0
 6680.0
 7380.0
 6380.0
 6540.0
 5440.0
 6400.0
 5880.0
 6380.0
 6320.0
 6140.0

In [299]:
split_ = 0.5
past = 3
num_past = 3

last_yT = false
max_cuts = 10
verbose = false

uncertainty = 0.#1
δ_inf = 0.02
ϵ_inf = 0.02

ϵ_l2 = 0.05
δ_l2 = 0.05

reg = 1/(past*num_past)
ρ = 100
val = 8; #n-split_index;

fix_β0 = false
more_data_for_β0 = false
benders = false
ridge = true

true

In [300]:
X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, max(split_index-num_past*past+1, 1), num_past*past, val, uncertainty, last_yT);

In [301]:
y0

10-element Vector{Float64}:
 -0.006859741651561132
  0.5487793321247895
 -0.8711871897481066
  0.5487793321247895
 -1.1181378892042624
  0.24009095780459472
 -1.6120392881165742
  1.4131067802213348
  0.8574677064449843
  3.018286326686348

In [302]:
X0

10×23 Matrix{Float64}:
 1.0  -6.7054  -6.17192  -5.47919  -5.81884  …  -6.59427  -6.82538  -6.0853
 1.0  -6.7054  -6.17192  -5.45864  -5.82076     -6.83569  -6.85181  -6.0853
 1.0  -6.7054  -6.17192  -5.43808  -5.82107     -4.95097  -5.37998  -6.0853
 1.0  -6.7054  -6.17192  -5.41752  -5.82113     -4.8312   -5.29895  -6.0853
 1.0  -6.7054  -6.17192  -5.39696  -5.82113     -6.68725  -6.95503  -6.0853
 1.0  -6.7054  -6.17192  -5.3764   -5.82113  …  -5.71148  -6.15402  -6.0853
 1.0  -6.7054  -6.17192  -5.35584  -5.82113     -6.2115   -6.12911  -6.0853
 1.0  -6.7054  -6.17192  -5.33528  -5.82113     -5.76942  -6.09456  -6.0853
 1.0  -6.7054  -6.17192  -5.31473  -5.82113     -4.91025  -4.47488  -6.0853
 1.0  -6.7054  -6.17192  -5.29417  -5.82113     -3.41126  -4.32448  -6.0853

In [314]:
β_list_linear_adaptive_trained_one = zeros(val, p)
#obj, β0_0, V0_0, β_list = adaptive_ridge_regression_exact_no_stable(X0, y0, ρ, ρ, past)
obj, β0_0, V0_0 = adaptive_ridge_regression_standard(X0, y0, 0.1, 10, past)

β_l2_init = l2_regression(X0,y0,0.1);

In [304]:
# X_ = vcat(X0,Xt)
# y_true = vcat(y0,yt)
# err_baseline = [abs(y_true[s]-dot(X_[s,:],β_l2_init)) for s=past+1:size(y_true)[1]]
# err_ARO = [abs(y_true[s]-dot(X_[s,:],β_list[s-past,:])) for s=past+1:size(y_true)[1]]

# get_metrics(err_baseline, y_true[past+1:end])

# println()
# get_metrics(err_ARO, y_true[past+1:end])

In [315]:
for s=1:val

        #TODO check split_index with max(split inex, 1)
        if more_data_for_β0
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, max(split_index-num_past*past+1, 1), s+(num_past-1)*past, past-1, uncertainty, last_yT)
        else
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, max(s+split_index-num_past*past+1, 1), (num_past-1)*past, past-1, uncertainty, last_yT)
        end

        #TODO: evaluate also if we change Vt regularly
        #obj, β_linear_adaptive_pure_0_Vt, Vt_adaptive_pure, _ = adaptive_ridge_regression_exact_Vt(vcat(X0,Xt), vcat(y0,yt), ρ, ρ, past, 1)


        #Line to get Z_{t+1}
        X_for_Z = X[split_index-past+s+1:split_index+s+1,:]
        X_for_Z[:,1] .= 1
        y_for_Z = y[split_index-past+s+1:split_index+s+1,:]
        X_, Z_test, y_ = get_X_Z_y(X_for_Z, y_for_Z, past)

        β_list_linear_adaptive_trained_one[s,:] = β0_0 + V0_0 * Z_test[1,:]
    end

In [316]:
X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, 1, split_index, val, uncertainty, last_yT);
yt_true = yt_true.*std_y.+mean_y
err_linear_adaptive_trained_one = [abs(yt_true[s]-(dot(Xt[s,:],β_list_linear_adaptive_trained_one[s,:]).*std_y.+mean_y)) for s=1:val]
err_baseline_test = [abs(yt_true[s]-(dot(Xt[s,:],β_l2_init).*std_y.+mean_y)) for s=1:val];


In [317]:
β0_0

23-element Vector{Float64}:
  0.0004953125958341266
 -0.003321267824075291
 -0.0030570308141640956
 -0.0013254587011632614
 -0.002883278389949345
 -0.0013254587011632614
 -0.0024219277318445107
  0.004161033227110593
  0.006584601491783848
  0.002542350885905425
  0.006116275303457392
  0.013878491957142011
 -0.0029675094805448055
 -0.009827123961388703
  0.009243632935233217
  0.009243632935233217
  0.006811936873750082
  0.002805746007153138
  0.0022986018205433103
  0.004052053569515718
  0.009936291400200473
  0.004763365116484622
 -0.0030141278484840522

In [318]:
get_metrics(err_baseline_test, yt_true)

println()
get_metrics(err_linear_adaptive_trained_one, yt_true)

MAE : 426.361165179312
MAPE : 7.102897225788196
RMSE : 504.77814059811254
R2 : -1.2238793037371845

MAE : 378.3884104339335
MAPE : 6.130978042081027
RMSE : 464.1517196799744
R2 : -0.8803126238872141


## Synthetic

In [9]:
N0 = 900
Nt = 100
p = 10

δ = 0.5
δ_pert = 0.5
σ_pert = 1
#y = randn(N0+Nt)
n0 = floor(Int, size(y_safi)[1]/2)
y_true = y_safi[n0-N0:n0+Nt-1]
y = copy(y_true)
d = Uniform(-δ, δ)
biases = rand(d, p)
variances = rand(p)
gradual_after = false
gradual_before = true

#Create the features X by taking y and adding noise.
X = zeros(N0+Nt, p)

for i=1:p
    d = Normal(biases[i], variances[i])
    X[:,i] = y.+ rand(d, N0+Nt)
end

#### Perturbations on base learners because of data drift
d = Normal(0, δ_pert)
biases_perturb = rand(d, p)
d = Uniform(0, σ_pert)
variances_perturb = rand(d, p)

if gradual_after
    for i=1:p
        d = Normal(biases_perturb[i], variances_perturb[i])
        #We add the perturbation for each model 
        #We make the perturbations more and more intense across time
        X[N0+1:N0+Nt,i] = X[N0+1:N0+Nt,i].+rand(d, Nt).*[t/Nt for t=1:Nt]
    end
end

if gradual_before
    for i=1:p
        for t=1:Nt
            #We compute the perturbation for each model 
            #We make the perturbations more and more intense across time
            d = Normal(t/Nt*δ_pert, t/Nt*σ_pert)
            X[N0+t,i] = X[N0+t,i]+rand(d)
        end
    end
else
    for i=1:p
        d = Normal(biases_perturb[i], variances_perturb[i])
        #We add the perturbation for each model 
        #We make the perturbations more and more intense across time
        X[N0+1:N0+Nt,i] = X[N0+1:N0+Nt,i].+rand(d, Nt)
    end
end

In [10]:
#### Perturbations on y
perturb_y = true
perturb_y_norm = true
y_bias = -0.1
y_var = 0.5

if perturb_y
    if perturb_y_norm
        d = Normal(y_bias, y_var)
        y[N0+1:N0+Nt] = y[N0+1:N0+Nt] .+ rand(d, Nt)
    else
        d = Uniform(-y_bias, y_bias)
        y[N0+1:N0+Nt] = y[N0+1:N0+Nt] .+ rand(d, Nt)
    end
end
;

In [29]:
split_ = N0/(N0+Nt)
past = 20
num_past = floor(Int, Nt/past)

last_yT = false
max_cuts = 10
verbose = false

uncertainty = 0.#1
δ_inf = 0.02
ϵ_inf = 0.02

ϵ_l2 = 0.05
δ_l2 = 0.05

reg = 1/(past*num_past)
ρ = 0.1
val = Nt-1; #n-split_index;

fix_β0 = false
more_data_for_β0 = false
benders = false
ridge = true

eval_method(X, y, y_true, split_, past, num_past, val, uncertainty, ϵ_inf, δ_inf, last_yT, 
        ϵ_l2, δ_l2, ρ, reg, max_cuts, verbose, 
        fix_β0, more_data_for_β0, benders, ridge)


### Mean Baseline ###
MAE : 0.42345111861420726
MAPE : 187.57814003246975
RMSE : 0.5017148222829678
R2 : 0.2684574206577698
CVAR 0.05 :1.1315406489739686
CVAR 0.15 :0.8671868300356882

### Bandits Full Baseline ###
MAE : 0.35048051123948665
MAPE : 163.8983188941456
RMSE : 0.5198624052014562
R2 : 0.21457889059766233
CVAR 0.05 :1.4609555545070267
CVAR 0.15 :1.1120710262538487

### Bandits Only Last T Baseline ###
MAE : 0.3664926282734096
MAPE : 176.46620187337854
RMSE : 0.5475155488397699
R2 : 0.1287984082660898
CVAR 0.05 :1.4955622544668161
CVAR 0.15 :1.1764829878349519

### Passive-Aggressive Baseline ###
MAE : 0.5103165968166972
MAPE : 193.79379347546438
RMSE : 0.6738334814289204
R2 : -0.3195653975318984
CVAR 0.05 :1.644262319552229
CVAR 0.15 :1.2777854831230044

### β0 Baseline ###
MAE : 0.21725233606432692
MAPE : 74.74218826820477
RMSE : 0.3017689156644273
R2 : 0.7353482619067152
CVAR 0.05 :0.8529027877143107
CVAR 0.15 :0.6180074087399314

### βt Linear Decision Rule Adaptive with 

# Linear rule

In [24]:
function adaptive_ridge_regression_standard(X, y, ρ_β0, ρ_V0, T)
    
#     Adaptive ridge: does not run fast
    

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)
    X, Z, y = get_X_Z_y(X, y, T)

    N, P = size(X)
    # Add variables

    @variable(model, β0[j=1:P])
    @variable(model, V0[j=1:P, k=1:T*P+T])
    @variable(model, t>=0)

    # Add objective
    @objective(model, Min, t + ρ_β0 * sum(β0[j]^2 for j=1:P)
                            + ρ_V0 * sum(V0[j,k]^2 for j=1:P for k=1:T*P+T)
    )

    @constraint(model, t >= 1 / N * sum((y[i] - sum(X[i, j] * (β0[j]
                 + sum(V0[j, l] * Z[i, l] for l=1:(T * P + T))
                 )
                for j=1:P))^2
        for i=1:N))

    optimize!(model);
    return objective_value(model), getvalue.(β0), getvalue.(V0), Z
end

adaptive_ridge_regression_standard (generic function with 1 method)

In [174]:
function adaptive_ridge_regression_before_exact(X, y, ρ_β0, ρ_V0, T)
    
    #Version with actual robust equivalence
    #We use beta t
    
    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 1)
    X, Z, y = get_X_Z_y(X, y, T)

    N, P = size(X)
    # Add variables

    @variable(model, β0[j=1:P])
    @variable(model, V0[j=1:P, k=1:T*P+T])
    @variable(model, t>=0)
    @variable(model, β[t=1:N,j=1:P])

    # Add objective
    @objective(model, Min, t + ρ_β0 * sum(β0[j]^2 for j=1:P)
                            + ρ_V0 * sum(V0[j,k]^2 for j=1:P for k=1:T*P+T)
    )

    @constraint(model, t >= 1 / N * sum((y[i] - sum(transpose(X[i, :])*β[i,:]))^2 for i=1:N))
        
    @constraint(model, [i=1:N], β[i,:] .== β0+V0*Z[i,:])

    optimize!(model);
    return objective_value(model), getvalue.(β0), getvalue.(V0), getvalue.(β)
end

adaptive_ridge_regression_before_exact (generic function with 1 method)

In [188]:
function adaptive_ridge_regression_exact(X, y, ρ_β0, ρ_V0, T, N0)
    
    #Version with actual robust equivalence
    #The formula for the regularization is different
    
    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)
    X, Z, y = get_X_Z_y(X, y, T)

    N, P = size(X)
    # Add variables

    @variable(model, β0[j=1:P])
    @variable(model, V0[j=1:P, k=1:T*P+T])
    @variable(model, t>=0)
    @variable(model, β[t=1:N,j=1:P])

    # If no stable part, then no reg
    if N0 == 1
        ρ_β0 = 0
    end
    
    # Add objective
    @objective(model, Min, t + 1/N0*ρ_β0 * sum(β0[j]^2 for j=1:P)
                            + 1/(N-N0)*ρ_V0 * sum(β[t,k]^2 for t=1:N for k=1:P)
    )

    @constraint(model, t >= 1 / N * sum((y[i] - sum(transpose(X[i, :])*β[i,:]))^2 for i=1:N))
    
    #stable and adaptive part
    @constraint(model, [i=N0:N], β[i,:] .== β0+V0*Z[i,:])
    @constraint(model, [i=1:N0-1], β[i,:] .== β0)

    optimize!(model);
    return objective_value(model), getvalue.(β0), getvalue.(V0), getvalue.(β)
end

adaptive_ridge_regression_exact (generic function with 2 methods)

In [12]:
function get_X_Z_y(X, y, T)
    """
    Input: training data X and corresponding labels y ; T: how many time-steps from the past to be used
    Output: the past features X with past targets y as a Z training data (no present features)
    """
    n, p = size(X)
    #T past time steps * p features + T targets
    Z = ones(n-T, T*p+T)
    for i=T+1:n
        for t=1:T
            Z[i-T,1+p*(t-1):p*t] = X[i-t,:]
        end
        Z[i-T, (p*T+1):end] = y[i-T:i-1]
    end
    return X[T+1:end,:], Z, y[T+1:end]
end

get_X_Z_y (generic function with 1 method)

In [13]:
e, Z, f = get_X_Z_y(X[1:5,1:2], y[1:5], 3)

([0.953041262548516 0.4782870249686575; 1.0535457742742587 0.334867774903859], [0.6507019593290054 0.4323663266396612 … -0.24864108988725883 0.4349146279684725; 0.953041262548516 0.4782870249686575 … 0.4349146279684725 0.7909332310183326], [0.7909332310183326, 0.7909332310183326])

In [164]:
obj, β0, V0, Z = adaptive_ridge_regression_standard(X[1:100,1:3], y[1:100], 0.1, 0.1, 3)
println(obj)

0.019454798289416986


In [165]:
β0+V0*Z[1,:]

3-element Vector{Float64}:
 0.1504249753800681
 0.4548341433068424
 0.14489314118672222

In [191]:
obj, β0, V0, β = adaptive_ridge_regression_exact(X[1:100,1:3], y[1:100], 0, 0.1, 3, 16)
println(obj)

0.5860827402650841


In [187]:
obj, β0, V0, β = adaptive_ridge_regression_exact(X[1:100,1:3], y[1:100], 0, 0.1, 3, 0)
println(obj)

0.5852809828471135


In [177]:
obj, β0, V0, β = adaptive_ridge_regression_before_exact(X[1:100,1:3], y[1:100], 0.1, 0.1, 3)
println(obj)
println(β0+V0*Z[1,:])
println(β[1,:])

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 291 rows, 331 columns and 4074 nonzeros
Model fingerprint: 0x6c09b65a
Model has 39 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e-02, 3e+00]
  QMatrix range    [1e-06, 8e-02]
  QLMatrix range   [6e-05, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e-01, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [8e-01, 8e-01]
Presolve time: 0.00s
Presolved: 391 rows, 432 columns, 4759 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 Dense cols : 41
 Free vars  : 291
 AA' NZ     : 4.854e+03
 Factor NZ  : 1.827e+04
 Factor Ops : 8.039e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual    

In [170]:
_, β0, V0, β, X1, Z1 = adaptive_ridge_regression_exact_test(X[1:100,1:3], y[1:100], 0.1, 0.1, 3)
β[1,:]

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 291 rows, 331 columns and 4074 nonzeros
Model fingerprint: 0xac638d92
Model has 39 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e-02, 3e+00]
  QMatrix range    [2e-01, 1e+01]
  QLMatrix range   [6e-01, 4e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e-01, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [8e-01, 8e-01]
Presolve removed 291 rows and 291 columns
Presolve time: 0.00s
Presolved: 42 rows, 83 columns, 864 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 8.600e+02
 Factor NZ  : 9.030e+02
 Factor Ops : 2.558e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual

3-element Vector{Float64}:
 0.1504249753800681
 0.45483414330684235
 0.14489314118672222

In [157]:
N, P = size(X1)
i = 1
T = 3
sum(X1[i, j] * (β0[j]+ sum(V0[j, l] * Z1[i, l] for l=1:(T * P + T))) for j=1:P)

0.533767797757831

In [151]:
transpose(X1[i,:])*β[i,:]

0.533767797757831

In [156]:
j = 1
println(β0[j]+sum(V0[j, l] * Z[i, l] for l=1:(T * P + T)))
println(β0+V0*Z[i,:])
println()

0.1504249753800681
[0.1504249753800681, 0.4548341433068424, 0.14489314118672222]


In [16]:
function evaluate_adaptive(X, y, y_true, β0, V0, T)
    X, Z, y = get_X_Z_y(X, y, T)
    
    N, P = size(X)

    pred = [sum(X[i, j] * (β0[j] + sum(V0[j, :] .* Z[i, :])) for j=1:P) for i=1:N]
    if size(y_true) == size(pred)
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    else
        y_true = y_true[T+1:end]
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    end
    return err
end

evaluate_adaptive (generic function with 1 method)

In [195]:
function evaluate_adaptive_retrained(X, y, y_true, β0_list, V0_list, T)
    X, Z, y = get_X_Z_y(X, y, T)
    
    N, P = size(X)

    pred = [sum(X[i, j] * (β0_list[i,j] + sum(V0_list[i, j, :] .* Z[i, :])) for j=1:P) for i=1:N]
    if size(y_true) == size(pred)
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    else
        y_true = y_true[T+1:end]
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    end
    return err
end

evaluate_adaptive_retrained (generic function with 1 method)

In [435]:
pred, err = evaluate_adaptive(X[1:100,1:3], y[1:100], y[4:100], β0, V0, 3)

([0.2772791481270517, -0.22139104249186187, 0.25353125721465, 0.13052228762534424, 0.16184232903639695, 0.02133275894336177, -0.23829147212091226, 0.14926772611852196, 0.28826028113700647, 0.7460128800839689  …  0.6974694989745849, 0.4334507651952026, 0.4535167025297599, 0.4228111533412082, -0.9470370564072363, -0.004111494312688896, 0.12274140243903106, 0.08108513878897414, -1.4025705199739362, 0.08251350395763898], [0.7111257347162706, 0.768332857553357, 0.1994216466416816, 0.6812973681052467, 0.08794262534614797, 0.24054297764516547, 0.02174221306394139, 0.4729100683816176, 0.04127284032870793, 0.5794229469798875  …  0.29188476681020137, 0.3992553152476475, 0.02696830662909544, 0.5024620796474668, 0.7050416471102421, 0.23408638321844036, 0.4254973199898675, 0.7075806393798176, 0.9146849288371715, 0.6178682421677187])

In [196]:
function eval_method(X, y, y_true, split_, past, num_past, val, uncertainty, ϵ_inf, δ_inf, last_yT,
        ϵ_l2, δ_l2, ρ, reg, max_cuts, verbose,
        fix_β0, more_data_for_β0, benders, ridge, linear_adapt)

    threshold_benders = 0.01
    n, p = size(X)
    split_index = floor(Int,n*split_)
    #TODO change spli_index with max(split inex, 1)
    X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, split_index-num_past*past+1, num_past*past, val, uncertainty, last_yT)

    β_list0 = zeros(val, p)
    β_listt = zeros(val, p)
    β_listl2 = zeros(val, p)
    β_l2_init = l2_regression(X0,y0,ρ);
    β0_list_linear_adapt = zeros(val, p)
    V0_list_linear_adapt = zeros(val, p, (past-1)*p+past-1)
    err = ones(val)
    
    ###Linear decision rule
    _, β0_0, V0_0 = adaptive_ridge_regression_standard(X0, y0, ρ, ρ, past-1)
    print(size(V0))
    
    for s=1:val
        #TODO Check how much I use from the past
        if more_data_for_β0
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, split_index-num_past*past+1, s+(num_past-1)*past, past-1, uncertainty, last_yT)
        else
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, s+split_index-num_past*past+1, (num_past-1)*past, past-1, uncertainty, last_yT)
        end

        if benders
            ##TODO handle fix_beta0
            obj, β0_val, α, y_val = master_problem(X0, Xt, y0, D_min, D_max, threshold_benders, ϵ_inf, δ_inf, reg, ρ, max_cuts, verbose)
            _, βt_val = S_primal(Xt, y_val, β0_val, ϵ_inf, δ_inf);
        else
            if ridge
                obj, βt_val, β0_val = master_primal_l2_ridge(X0, Xt, y0, D_min, D_max, ϵ_inf, δ_inf, reg, ρ, ϵ_l2, δ_l2, fix_β0, β_l2_init)
            else
                obj, βt_val, β0_val = master_primal_l2(X0, Xt, y0, D_min, D_max, ϵ_inf, δ_inf, reg, ρ, ϵ_l2, δ_l2, fix_β0, β_l2_init)
            end
        end
        
        if linear_adapt
            #err[s] = evaluate_adaptive(vcat(X0[end-past+2:end,:],Xt), vcat(y0[end-past+2:end,:],yt), yt_true, β0, V0, past)
            _, β0, V0 = adaptive_ridge_regression_standard(vcat(X0,Xt), vcat(y0,yt), ρ, ρ, past-1)
            β0_list_linear_adapt[s,:] = β0
            V0_list_linear_adapt[s,:,:] = V0
            N = size(X0)[1]
        end
        
        β_listt[s,:] = βt_val[past-1,:]
        β_list0[s,:] = β0_val
        β_l2 = l2_regression(vcat(X0,Xt),vcat(y0,yt),ρ);
        β_listl2[s,:] = β_l2

    end

    X0, y0, Xt, yt, _, D_min, D_max = prepare_data_from_y(X, y, 1, split_index, val, uncertainty, last_yT)
    _, _, _, _, yt_true, _, _ = prepare_data_from_y(X, y_true, 1, split_index, val, uncertainty, last_yT)

    N = size(X0)[1]
    err_linear_rule = evaluate_adaptive(vcat(X0[N-past+2:end,:],Xt), vcat(y0[N-past+2:end,:],yt), yt_true, β0_0, V0_0, past-1)
    err_linear_rule_retrained = evaluate_adaptive_retrained(vcat(X0[N-past+2:end,:],Xt), vcat(y0[N-past+2:end,:],yt), 
        yt_true, β0_list_linear_adapt, V0_list_linear_adapt, past-1)
    
    err_0 = [abs(yt_true[s]-dot(Xt[s,:],β_list0[s,:])) for s=1:val]
    err_t = [abs(yt_true[s]-dot(Xt[s,:],β_listt[s,:])) for s=1:val]
    err_baseline = [abs(yt_true[s]-dot(Xt[s,:],β_l2_init)) for s=1:val]
    err_l2 = [abs(yt_true[s]-dot(Xt[s,:],β_listl2[s,:])) for s=1:val]

    println("\n### β0 Baseline ###")
    println("MAE Baseline: ", mean(err_baseline))
    println("CVAR 0.05 :", compute_CVaR(err_baseline, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_baseline, 0.15))
    println("R2 : ", R2_err(err_baseline, yt_true))

    println("\n### β0 Baseline Retrained ###")
    println("MAE Baseline: ", mean(err_l2))
    println("CVAR 0.05 :", compute_CVaR(err_l2, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_l2, 0.15))
    println("R2 : ", R2_err(err_l2, yt_true))
    
    println("\n### β0 V0 Linear rule Adaptive ###")
    println("MAE Baseline: ", mean(err_linear_rule))
    println("CVAR 0.05 :", compute_CVaR(err_linear_rule, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_linear_rule, 0.15))
    println("R2 : ", R2_err(err_linear_rule, yt_true))
    
    println("\n### β0 V0 Linear rule Adaptive Retrained ###")
    println("MAE Baseline: ", mean(err_linear_rule_retrained))
    println("CVAR 0.05 :", compute_CVaR(err_linear_rule_retrained, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_linear_rule_retrained, 0.15))
    println("R2 : ", R2_err(err_linear_rule_retrained, yt_true))

    println("\n### β0 Adaptive ###")
    println("MAE 0: ", mean(err_0))
    println("CVAR 0.05 :", compute_CVaR(err_0, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_0, 0.15))
    println("R2 : ", R2_err(err_0, yt_true))

    println("\n### βt Adaptive ###")
    println("MAE t: ", mean(err_t))
    println("CVAR 0.05 :", compute_CVaR(err_t, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_t, 0.15))
    println("R2 : ", R2_err(err_t, yt_true))
end

eval_method (generic function with 2 methods)

In [198]:
linear_adapt = true
split_ = N0/(N0+Nt)
past = 5
num_past = 20#floor(Int, Nt/past)

last_yT = false
max_cuts = 10
verbose = false

uncertainty = 0.#1
δ_inf = 0.02
ϵ_inf = 0.02

ϵ_l2 = 0.05
δ_l2 = 0.05

reg = 1/(past*num_past)
ρ = 0.1
val = Nt-1; #n-split_index;

fix_β0 = false
more_data_for_β0 = false
benders = false
ridge = true

eval_method(X, y, y_true, split_, past, num_past, val, uncertainty, ϵ_inf, δ_inf, last_yT, 
        ϵ_l2, δ_l2, ρ, reg, max_cuts, verbose, 
        fix_β0, more_data_for_β0, benders, ridge, linear_adapt)

(3, 12)

LoadError: Gurobi Error 10020: Constraint Q not PSD (diagonal adjustment of 1.1e-05 would be required). Set NonConvex parameter to 2 to solve model.